# Проект по SQL


# Цель исследования

**Проанализировать базу данных крупного сервиса для чтения книг по подписке.**

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут в дальнейшем сформулировать ценностное предложение для нового продукта.

# Задачи исследования

- Посчитать, сколько книг вышло после 1 января 2000 года
- Для каждой книги посчитать количество обзоров и среднюю оценку
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры)
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

# Описание данных

**Таблица books**

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.


## Исследуем таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# таблица с книгами
query = '''
SELECT *
FROM books
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# таблица с авторами
query = '''
SELECT *
FROM authors
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# таблица с издательствами
query = '''
SELECT *
FROM publishers
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# таблица с оценками
query = '''
SELECT *
FROM ratings
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# таблица с отзывами
query = '''
SELECT *
FROM reviews
LIMIT 5;
                    '''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитаем, сколько книг вышло после 1 января 2000 года


In [7]:
query = '''
SELECT 
    COUNT(book_id) AS count_book
FROM 
    books
WHERE 
    publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine) 

,count_book
0,819


После 1 января 2000 года вышло 819 книг

## Для каждой книги посчитаем количество обзоров и среднюю оценку


In [8]:
query = '''
SELECT 
    books.book_id, 
    books.title, 
    COUNT(DISTINCT reviews.review_id) AS review_count, 
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM 
    books 
        LEFT JOIN reviews ON books.book_id = reviews.book_id
        LEFT JOIN ratings ON books.book_id = ratings.book_id
    
GROUP BY books.book_id
ORDER BY review_count DESC
'''
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Для каждой книги посчитали количество обзоров(review_count) и среднюю оценку(avg_rating)

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить брошюры)

In [9]:
query = '''
SELECT publishers.publisher, COUNT(books.book_id) AS books_count
FROM 
    publishers 
        LEFT JOIN books ON books.publisher_id = publishers.publisher_id
    
WHERE books.num_pages > 50    
GROUP BY publishers.publisher
ORDER BY books_count DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine) 

,publisher,books_count
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц - это Penguin Books, издавшее 42 книги

## Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками


In [10]:
query = '''
SELECT authors.author, ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM 
    books 
        LEFT JOIN authors ON books.author_id = authors.author_id
        LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE
    books.book_id IN (
        SELECT ratings.book_id
        FROM ratings
        GROUP BY ratings.book_id
        HAVING COUNT(ratings.rating) > 50)
GROUP BY authors.author
 
ORDER BY avg_rating DESC
LIMIT 3

'''
pd.io.sql.read_sql(query, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.25


Автор с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками - это J.K. Rowling/Mary GrandPré со средним рейтингом 4.29

Вполне ожидаемо :)

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query = '''
SELECT ROUND(AVG(t1.count_review), 2) AS avg_reviews 
FROM (

    SELECT reviews.username, COUNT(reviews.review_id) AS count_review
        FROM reviews
    WHERE username IN (

                    SELECT ratings.username
                    FROM 
                        ratings
  
                    GROUP BY ratings.username
                    HAVING COUNT(ratings.rating_id) > 50
                        )
    GROUP BY reviews.username
    ) AS t1

'''
pd.io.sql.read_sql(query, con = engine) 

,avg_reviews
0,24.33


Пользователи, которые поставили больше 50 оценок, в среднем оставляют 24.3 обзора

# Вывод

- После 1 января 2000 года вышло 819 книг
- Для каждой книги посчитали количество обзоров(review_count) и среднюю оценку(avg_rating)
- Издательство, которое выпустило наибольшее число книг толще 50 страниц - это Penguin Books, издавшее 42 книги
- Автор с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками - это J.K. Rowling/Mary GrandPré со средним рейтингом 4.29

- Пользователи, которые поставили больше 50 оценок, в среднем оставляют 24.3 обзора